# KNN

### Python Library ...

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from matplotlib import pyplot as plt


### Load All The Data....

In [11]:
# Load the data
file_path = '../New_Data/Final_Dataset.xlsx'  # Replace with your file path
data_01 = pd.read_excel(file_path)

### Creating Synthetic Data ....

In [36]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tsaug import AddNoise, TimeWarp, Drift

# Original dataset (features only)
Data = data_01.loc[:, 'ROM_angle':90].values 

# Number of synthetic samples to generate
num_synthetic_samples = 20

# Extract Min and Max ROM angle from the original data
rom_angle_min = Data[:, 0].min()
rom_angle_max = Data[:, 0].max()

# Function to constrain ROM angle within original Min and Max range
def constrain_rom_angle(augmented_sample, rom_angle_min, rom_angle_max):
    augmented_sample[:, 0] = np.clip(augmented_sample[:, 0], rom_angle_min, rom_angle_max)
    return augmented_sample

# Augmentation pipeline
augmenter = (
    TimeWarp(n_speed_change=1, max_speed_ratio=1.2) +
    AddNoise(scale=0.005) +
    Drift(max_drift=0.01, n_drift_points=2)
)
# Generate synthetic samples
synthetic_samples = []
for _ in range(num_synthetic_samples):
    sampled_rows = Data[np.random.choice(Data.shape[0], size=1, replace=False)]
    augmented_sample = augmenter.augment(sampled_rows)
    augmented_sample = constrain_rom_angle(augmented_sample, rom_angle_min, rom_angle_max)
    synthetic_samples.append(augmented_sample)

# Combine original and synthetic samples
synthetic_samples = np.vstack(synthetic_samples)
augmented_dataset = np.vstack([Data, synthetic_samples])

data = pd.DataFrame(augmented_dataset)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,74.0,310.000000,437.034965,518.473193,562.940559,579.062937,575.466200,562.794872,552.722611,548.293706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,150.0,1371.293706,1259.902098,1244.452214,1302.843823,1412.976690,1552.750583,1676.762238,1789.277389,1886.198135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,157.0,1842.937063,1915.902098,1990.107226,2060.264569,2121.086247,2167.284382,2189.783217,2182.948718,2142.102564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,155.0,1605.195804,1746.097902,1851.216783,1925.229604,1972.813520,1998.645688,2017.405594,2005.682984,1985.494172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,160.0,2072.496503,2132.825175,2151.382284,2141.973193,2118.403263,2094.477855,2103.848485,2135.745921,2196.694639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Extract EEG features and target...

In [37]:
# Extract EEG features and target
X = data.loc[:, 1:181].values  # EEG data (columns D to MI) iloc[:,3:]
y = data.loc[:, 0].values  # Target ROM angle (column C)
max_rom = y.max()
min_rom = y.min()
# Check for extreme values in target
print(f"Max ROM Angle: {max_rom}, Min ROM Angle: {min_rom}")

# Normalise the target variable
y = (y - min_rom) / (max_rom - min_rom)


Max ROM Angle: 160.0, Min ROM Angle: 58.0


### Normalise the features...

In [38]:
# Normalise the features
scaler = MinMaxScaler()
X_normalised = scaler.fit_transform(X)

# Reshape X for GRU (samples, timesteps, features)
n_features = X_normalised.shape[1]
X_reshaped = X_normalised.reshape(X_normalised.shape[0], 1, n_features)

In [39]:
reshaped_array = X_reshaped.squeeze(axis=1)  # Remove the second dimension
print(reshaped_array.shape)

(141, 181)


### Split data into training and testing sets ...

In [40]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.1, random_state= 4)

In [41]:
import numpy as np

# Reshape to 2D
X_X = X_train.squeeze(axis=1)
X_Xtest = X_test.squeeze(axis=1)

print(X_X)

[[1.00000000e+00 9.39116529e-01 8.83633845e-01 ... 6.53860181e-01
  6.66383740e-01 6.83099657e-01]
 [9.40684698e-01 1.00000000e+00 9.98595138e-01 ... 1.40794801e-02
  1.93586079e-02 2.10371033e-02]
 [4.91045829e-01 5.20422962e-01 5.42388226e-01 ... 1.40794801e-02
  1.93586079e-02 2.10371033e-02]
 ...
 [5.89724108e-01 5.81945948e-01 5.91236259e-01 ... 1.40794801e-02
  1.93586079e-02 2.10371033e-02]
 [5.56837563e-01 5.51343498e-01 5.52964393e-01 ... 4.55836836e-04
  9.19606817e-03 8.51322266e-03]
 [1.13726026e-01 1.80218033e-02 1.00464572e-02 ... 1.36614046e-02
  1.82309405e-02 2.23803991e-02]]


### KNN Model...

In [42]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=1)
model.fit(X_X, y_train)
predictions_KNN = model.predict(X_Xtest)

In [43]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Make predictions
predictions_knn = model.predict(X_Xtest)

r2 = r2_score(y_test,predictions_knn )
mae = mean_absolute_error(y_test,predictions_knn )
mse = mean_squared_error(y_test,predictions_knn )

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 93.319687%
MAE : 0.039647
MSE : 0.002861


### Invers The Metrix...

In [29]:
# Function for invers metrixes...
def invers_prediction(predic):
    prz = predic
    Predictions = []
    for i in range(prz.shape[0]):
        pred = prz[i]*(max_rom-min_rom) + min_rom
        Predictions.append(pred)

    return Predictions


In [44]:
inv_predicted = invers_prediction(predictions_knn )
inver_y_test = invers_prediction(y_test)


### R2 Score for Invers metrixes...

In [45]:
r2 = r2_score(inver_y_test,inv_predicted)
mae = mean_absolute_error(inver_y_test,inv_predicted)
mse = mean_squared_error(inver_y_test,inv_predicted)

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 93.319687%
MAE : 4.043956
MSE : 29.763508


In [32]:
pd_result_table = pd.DataFrame(data = [inver_y_test,inv_predicted], index = ["y-test","prd"])
pd_result_table

,0,1,2,3,4,5,6,7,8,9,10,11,12
y-test,58.0,157.0,116.0,64.0,126.0,150.0,149.0,98.0,160.0,96.0,160.0,158.0,100.0
prd,65.0,154.0,129.0,62.0,125.0,151.0,146.0,99.0,156.0,99.0,156.0,154.0,102.0
